In [1]:
!pip install icalendar


[notice] A new release of pip available: 22.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import json
from pathlib import Path
import hashlib
from icalendar import Calendar, Event
from datetime import datetime, date, timedelta, timezone

In [3]:
working_dir = Path("../json/")

In [4]:
def generate_uid(name, date, state_code):
    unique_string = f"{date}-{name}"
    sha1_hash = hashlib.sha1(unique_string.encode('utf-8')).hexdigest()
    return f"{state_code}-{sha1_hash[:12]}@feiertage.ics.tools"

In [5]:
# Feiertage einlesen und ics Datei generieren
public_holidays_dir = Path("../json/feiertage/")
target_public_holidays_dir = Path("../Feiertage/")
if not target_public_holidays_dir.exists():
    target_public_holidays_dir.mkdir()

for json_datei in public_holidays_dir.glob('*.json'):
    with open(json_datei, 'r', encoding='utf-8') as f:
        daten = json.load(f)
    
    # Prüfen ob Dateinamen mit den Metadaten übereinstimmt
    federal_state = daten.get('metadata').get('state')
    federal_state_iso_code = daten.get('metadata').get('code')
    erwarteter_name = f"{federal_state.lower()}.json"
    tatsaechlicher_name = json_datei.name.lower()
    
    if erwarteter_name != tatsaechlicher_name:
        raise RuntimeError(
            f"Validierungsfehler: Dateiname '{json_datei.name}' stimmt nicht mit "
            f"Bundesland '{daten.get('metadata').get('state')}' überein (erwartet: '{erwarteter_name}')."
        )
    
    # Generiere Kalender
    cal = Calendar()
    cal.add('prodid', '-//ics.tools//ics.tools Feiertage v1.0//DE')
    cal.add('version', '2.0')
    cal.add('x-wr-calname', f'{federal_state} Feiertage')
    cal.add('method', 'PUBLISH')
    
    # Füge Feiertage hinzu
    for item in daten.get('holidays'):
        event = Event()
        event.add('summary', item.get('name'))
        start_date = datetime.fromisoformat(item['date']).date()
        end_date = start_date + timedelta(days=1)
        event.add('dtstart', start_date)
        event.add('dtend', end_date)
        event.add('dtstamp', datetime.now(timezone.utc))
        event.add('uid', generate_uid(item.get('name'), start_date, federal_state_iso_code))
        cal.add_component(event)

    with open(target_public_holidays_dir / f"{federal_state.lower()}.ics", 'wb') as f:
        f.write(cal.to_ical())
